# Demostration of Different Predictors and Recommenders

## Sample Data Generation

### Toy Example Data

In [10]:
from scipy.sparse import csr_matrix
user_ratings = csr_matrix([
    [5, 4, 5, 3, 3],
    [3, 2, 2, 4, 1],
    [3, 4, 3, 5, 4],
    [5, 1, 4, 2, 4],
    [2, 3, 4, 1, 1],
    [2, 3, 4, 2, 5],
])

test_set = [
    (0, 0),
    (0, 3),
    (1, 1),
    (1, 4),
    (2, 0),
    (2, 4),
    (3, 2),
    (4, 1),
    (4, 3),
    (5, 0),
]
training_set = []
for i in range(user_ratings.shape[0]):
    for j in range(user_ratings.shape[1]):
        if (i, j) not in test_set and user_ratings[i, j] != 0:
            training_set.append((i, j))

In [4]:
from scipy.sparse import csr_matrix

user_ratings = csr_matrix(
    [
        [5, 4, 4, 0, 5],
        [0, 3, 5, 3, 4],
        [5, 2, 0, 2, 3],
        [0, 2, 3, 1, 2],
        [4, 0, 5, 4, 5],
        [5, 3, 0, 3, 5],
        [3, 2, 3, 2, 0],
        [5, 3, 4, 0, 5],
        [4, 2, 5, 4, 0],
        [5, 0, 5, 3, 4],
    ]
)
test_set = [
    (0, 4),
    (1, 3),
    (2, 3),
    (3, 1),
    (4, 2),
    (5, 0),
    (6, 1),
    (7, 1),
    (8, 0),
    (9, 0),
]
training_set = []
for i in range(user_ratings.shape[0]):
    for j in range(user_ratings.shape[1]):
        if (i, j) not in test_set and user_ratings[i, j] != 0:
            training_set.append((i, j))

### Random Data

In [1]:
from samples import generate_sample_data
import json

sample_data = generate_sample_data(1000, 1000, 200)
with open("samples/data.json", "w") as f:
    json.dump(sample_data, f)

In [2]:
import json
from random import randrange
with open("samples/data.json", "r") as f:
    sample_data = json.load(f)

user_ratings = sample_data["ratings"]
test_set_size = int(len(user_ratings) * len(user_ratings[0]) * 0.2)
test_set = [(randrange(0, len(user_ratings)), randrange(0, len(user_ratings[0]))) for _ in range(test_set_size)]

### Real Datasets (from MovieLens)

In [1]:
import requests
import zipfile
import io

# Download the latest (small) dataset
url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
response = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(response.content))

# Unzip the dataset into a folder
z.extractall("data/")

# Download the latest (full) dataset
url = "https://files.grouplens.org/datasets/movielens/ml-latest.zip"
response = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(response.content))

# Unzip the dataset into a folder
z.extractall("data/")

In [1]:
import pandas as pd
from random import sample
from scipy.sparse import csr_matrix

print("Reading CSV files...")
# Read the ratings and movies CSV
# ratings_df = pd.read_csv("data/ml-latest-small/ratings.csv")
# movies_df = pd.read_csv("data/ml-latest-small/movies.csv")

# Read the ratings and movies CSV (WARNING: FULL DATASET)
ratings_df = pd.read_csv("data/ml-latest/ratings.csv")
movies_df = pd.read_csv("data/ml-latest/movies.csv")

# Convert the CSV into a user ratings table
# Create a dense matrix where each row represents a user and each column a movie.
# Missing ratings are filled with 0.
print("Pivotting data...")
user_ids = sorted(ratings_df["userId"].unique())
movie_ids = sorted(ratings_df["movieId"].unique())
user_id_map = {uid: i for i, uid in enumerate(user_ids)}
movie_id_map = {mid: j for j, mid in enumerate(movie_ids)}

rows = ratings_df["userId"].map(user_id_map).values
cols = ratings_df["movieId"].map(movie_id_map).values
data = ratings_df["rating"].values

user_ratings = csr_matrix((data, (rows, cols)), shape=(len(user_ids), len(movie_ids)))
movie_id_mappings = movies_df["movieId"].to_list()

print(
    "User ratings table created with dimensions:",
    user_ratings.shape[0],
    "rows x",
    user_ratings.shape[1],
    "columns",
)

print("Making test sets...")
# Get all indices with an existing (non zero) rating
valid_entries = list(zip(*user_ratings.nonzero()))
shuffuled_valid_entries = sample(valid_entries, k=len(valid_entries))
test_set_size = int(len(valid_entries) * 0.2)

# Randomly select test_set_size indices from the valid entries
test_set = shuffuled_valid_entries[:test_set_size]
training_set = shuffuled_valid_entries[test_set_size:]
print("Test set created with size:", len(test_set))
print("Done.")

Reading CSV files...
Pivotting data...
User ratings table created with dimensions: 330975 rows x 83239 columns
Making test sets...
Test set created with size: 6766432
Done.


In [2]:
from utils import get_test_set_matrix, remove_test_set

training_data = remove_test_set(user_ratings, test_set)
test_data = get_test_set_matrix(user_ratings, test_set)

Removing test set entries from training data...


100%|██████████| 305844/305844 [00:07<00:00, 40596.77it/s]


Done removing test set entries.
Getting test set matrix...


100%|██████████| 305844/305844 [00:09<00:00, 32264.19it/s]


Done getting test set matrix.


## Rating Predictors

### Least Squares Optimiation Predictor (Baseline)

In [3]:
from predictors.least_squares import LeastSquaresPredictor
from utils import root_mean_square_error_entries


baseline = LeastSquaresPredictor(shape=user_ratings.shape, lmda=0.2)
baseline.iterative_train(training_data=training_data)
test_predictions = baseline.predict(test_set)
training_predictions = baseline.predict(training_set)
# print(f"{test_predictions = }")
# print(f"{training_predictions = }")
# print(f"{training_data.data = }")
rmse_training = root_mean_square_error_entries(training_predictions, training_set, training_data)
rmse_test = root_mean_square_error_entries(test_predictions, test_set, test_data)
print(f"{rmse_training = }")
print(f"{rmse_test = }")

Constructing relevant matrices for iterative training...
Calculating user and item biases using Conjugate Gradient...


 11%|█▏        | 1130/10000 [01:10<09:16, 15.94it/s]


Training done.
Predicting entries...


100%|██████████| 6766432/6766432 [00:28<00:00, 237645.60it/s]


Finished predicting entries.
Predicting entries...


100%|██████████| 27065730/27065730 [03:07<00:00, 143967.44it/s]


Finished predicting entries.
Converting predictions to csr_matrix...
Calculating errors...
Calculating sum of square of errors...
Converting predictions to csr_matrix...
Calculating errors...
Calculating sum of square of errors...
rmse_training = np.float64(0.8476326427520402)
rmse_test = np.float64(0.8623229504665875)


In [14]:
baseline.train(training_data=training_data)
test_predictions = baseline.predict(test_set)
training_predictions = baseline.predict(training_set)
print(f"{test_predictions = }")
print(f"{training_predictions = }")
print(f"{training_data.data = }")
rmse_training = root_mean_square_error_entries(training_predictions, training_set, training_data)
rmse_test = root_mean_square_error_entries(test_predictions, test_set, test_data)
print(f"{rmse_training = }")
print(f"{rmse_test = }")

Constructing relevant matrices for iterative training...
Calculating user and item biases using Conjugate Gradient...
Training done.
Predicting entries...


100%|██████████| 20167/20167 [00:00<00:00, 2033378.90it/s]


Finished predicting entries.
Predicting entries...


100%|██████████| 80669/80669 [00:00<00:00, 1995225.32it/s]

Finished predicting entries.
test_predictions = array([3.23165064, 2.24782501, 4.16818974, ..., 3.83512761, 3.45890829,
       3.48633018], shape=(20167,))
training_predictions = array([3.17870269, 3.55298002, 3.47077656, ..., 3.20558118, 2.99453068,
       4.42682523], shape=(80669,))
training_data.data = array([4., 4., 4., ..., 5., 5., 3.], shape=(80669,))
Converting predictions to csr_matrix...
Calculating errors...
Calculating sum of square of errors...
Converting predictions to csr_matrix...
Calculating errors...
Calculating sum of square of errors...
rmse_training = np.float64(0.7776322133908681)
rmse_test = np.float64(0.880068881467453)


### Neighbor Correlations Predictor (based on Least Sqaures Optimization) (Improved)
As this requires calculating cosine coefficient for every single pair of items (or users, depending on the correlation chosen), this is extremely computationally expensive. Not recommended for data with a large number of items (or users).

In [3]:
from predictors.neighbor_correlations import Correlation, NeighborCorrelationsPredictor
from utils.neighbor_selection import most_similar, two_most_similar_skip_masked, two_most_similar

# baseline.predict_all = lambda quiet=False: np.array(
#     [
#         [np.nan, 2.7, 3.3, np.nan, 4.5],
#         [4.1, np.nan, 3.5, 4.9, np.nan],
#         [np.nan, 3.8, 2.5, 4.2, np.nan],
#         [2.8, 3.1, np.nan, 2.6, 4.8],
#         [3.3, np.nan, 3.7, np.nan, 2.4],
#         [np.nan, 3.9, 4.0, 1.5, 3.9],
#     ]
# )

improved = NeighborCorrelationsPredictor(baseline=baseline, correlation=Correlation.USER)
improved.train(training_data, most_similar)
test_predictions = improved.predict(test_set)
training_predictions = improved.predict(training_set)
print(f"{test_predictions = }")
print(f"{training_predictions = }")

Calculating cosine similarity coefficients...
Making neighbor table...


100%|██████████| 9724/9724 [00:09<00:00, 1035.61it/s]


Finished training.
Predicting entries...


100%|██████████| 20167/20167 [00:00<00:00, 89265.66it/s]


Finished predicting entries.
Predicting entries...


100%|██████████| 80669/80669 [00:00<00:00, 93488.19it/s]

Finished predicting entries.
test_predictions = array([2.6100543 , 1.53398421, 3.19146798, ..., 3.68492274, 3.60581692,
       3.53463773], shape=(20167,))
training_predictions = array([2.20713821, 5.        , 5.        , ..., 3.78632189, 2.49114979,
       4.06934193], shape=(80669,))


### Latent Factor Predictor

In [17]:
from scipy.sparse import csr_matrix

user_ratings = csr_matrix(
    [
        [3, 4, 5, 3, 2, 3],
        [3, 2, 3, 4, 2, 1],
        [4, 4, 4, 5, 3, 2],
        [3, 5, 4, 4, 3, 4],
        [2, 1, 2, 2, 3, 1],
        [3, 5, 5, 4, 4, 3],
        [3, 5, 5, 3, 2, 2],
        [2, 3, 3, 2, 1, 2],
    ]
)
test_set = [
    (0, 0),
    (1, 1),
    (2, 3),
    (2, 4),
    (3, 0),
    (3, 1),
    (5, 1),
    (5, 4),
    (6, 0),
    (6, 2),
    (7, 1),
    (7, 3),
]
training_set = []
for i in range(user_ratings.shape[0]):
    for j in range(user_ratings.shape[1]):
        if (i, j) not in test_set and user_ratings[i, j] != 0:
            training_set.append((i, j))

In [8]:
import numpy as np
from predictors.latent_factor import LatentFactorPredictor

u, i = training_data.shape
k = 5
latent = LatentFactorPredictor(
    shape=training_data.shape,
    k=k,
    p=np.ones(shape=(k,u), dtype=np.float64),
    q=np.ones(shape=(k,i), dtype=np.float64),
    lmda=0.2,
)
print(f"{latent.p = }")
print(f"{latent.q = }")
t = 0

latent.p = array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], shape=(5, 330975))
latent.q = array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], shape=(5, 83239))


In [9]:
latent.train(training_data=training_data, max_iterations=10000, tol=1e-4)
# t += 20
# print(f"Total: {t} iterations")

Preparing data for training...
Performing alternating least squares...


 37%|███▋      | 3681/10000 [5:20:56<9:10:56,  5.23s/it] 

Finished training.


In [10]:
training_predictions = latent.predict(training_set)
test_predictions = latent.predict(test_set)
print(f"{training_predictions = }")
print(f"{test_predictions = }")
print(f"{latent.p = }")
print(f"{latent.q = }")

Predicting entries...


100%|██████████| 27065730/27065730 [06:09<00:00, 73342.97it/s] 


Finished predicting entries.
Predicting entries...


100%|██████████| 6766432/6766432 [01:17<00:00, 87283.35it/s] 

Finished predicting entries.
training_predictions = array([4.03971698, 4.01497427, 3.69831421, ..., 2.98674756, 4.31886696,
       4.63071298], shape=(27065730,))
test_predictions = array([3.86332547, 2.69715931, 3.00076724, ..., 3.92449487, 2.50743115,
       4.86522182], shape=(6766432,))
latent.p = array([[-0.07819721, -0.19603844,  0.41203395, ...,  0.05589664,
        -0.19264802, -0.03581926],
       [ 0.51874046,  0.32971636,  0.54947275, ...,  0.64599256,
         0.42612923,  0.42105711],
       [ 0.2079426 ,  0.60165348,  0.53124531, ...,  0.50344707,
         0.88613082,  0.81888573],
       [ 0.9978182 ,  0.56970499,  1.21541057, ...,  0.99389135,
         0.40230275,  0.36734866],
       [ 0.54536394,  0.84827681,  0.71448997, ...,  0.06810958,
         0.34297778,  0.10050336]], shape=(5, 330975))
latent.q = array([[-1.36015647, -0.30197972, -0.22320699, ...,  0.73504535,
         0.41572854,  0.78204922],
       [-0.27344002,  0.4465125 ,  0.48633358, ...,  0.61678923,
 

In [11]:
from utils import root_mean_square_error_entries, root_mean_square_error

rmse_test = root_mean_square_error_entries(test_predictions, test_set, test_data)
print(f"{rmse_test = }")
rmse_training = root_mean_square_error_entries(training_predictions, training_set, training_data)
print(f"{rmse_training = }")

Gathering entries from predictions...


  0%|          | 0/6766432 [00:00<?, ?it/s]

100%|██████████| 6766432/6766432 [01:00<00:00, 111047.18it/s]


rmse_test = np.float64(0.8169738957584134)
Gathering entries from predictions...


100%|██████████| 27065730/27065730 [04:37<00:00, 97430.13it/s] 


rmse_training = np.float64(0.7484386486325613)


In [12]:
import pickle

with open('models/latent_' + str(k), 'wb') as f:
    pickle.dump(latent, f)

## Making Recommendations

### Model Import

In [22]:
import pickle

with open('models/latent_5', 'rb') as f:
    latent = pickle.load(f)

### Plain Recommender

In [19]:
from recommenders.plain import PlainRecommender

recommender = PlainRecommender(
    predictor=latent, users=user_ratings.shape[0], items=user_ratings.shape[1]
)

### Pure Score Boost

In [ ]:
import random
from recommenders.rating_boost import ScoreBoostRecommender

bids = [
    (idx, random.random()) for idx in random.sample(range(user_ratings.shape[1]), k=50)
]
paid_recommender = ScoreBoostRecommender(
    predictor=latent,
    users=user_ratings.shape[0],
    items=user_ratings.shape[1],
    bids=bids,
    alpha=0.1,
    beta=50,
    promotion_slots=[True if x % 4 == 0 else False for x in range(20)]
)
print("Bids:", sorted(bids, reverse=True, key=lambda x: x[1]))

Bids: [(46942, 0.9775303441841998), (21347, 0.9727706273992698), (71436, 0.9701877911829461), (61070, 0.9405803813160011), (74669, 0.9283088788729467), (50598, 0.9241025441836925), (73675, 0.9097545780350953), (17335, 0.9048744484425411), (14248, 0.8999265667689224), (31657, 0.880436228722251), (52392, 0.8762451549359666), (47340, 0.821007160082798), (67945, 0.7586887266949272), (46418, 0.7360430112066524), (40308, 0.7141250732929876), (49083, 0.678510780285214), (41445, 0.6738304390613612), (44428, 0.6709360394959633), (66159, 0.6556073094298421), (50346, 0.6425003723243433), (76690, 0.6218229040324436), (47605, 0.6139214073516629), (44797, 0.571536067266591), (1282, 0.5349779633031715), (14118, 0.5103750079380108), (55166, 0.49592736841765284), (51226, 0.4909192653260308), (64491, 0.4840806142975823), (58792, 0.484033915515084), (47784, 0.43125369444671335), (35692, 0.4149874906543358), (7416, 0.38077117121015536), (2038, 0.34273111850868154), (63649, 0.28242042024265135), (57630, 0.

In [21]:
from random import randint

# print(recommender.users, recommender.items)
# user = randint(0, user_ratings.shape[0])
user = 158336
print("Without promotion:", user, [movie_id_mappings[x] for x in recommender.recommend_items(user, 20)])
print("With promotion:", user, [movie_id_mappings[x] for x in paid_recommender.recommend_items(user, 20)])

Without promotion: 158336 [87700, 231059, 203427, 189087, 227901, 215003, 227913, 218125, 257751, 209717, 188597, 189143, 257811, 270476, 188033, 148677, 170835, 211225, 172727, 267538]
With promotion: 158336 [158242, 203427, 87700, 257751, 138446, 170835, 231059, 189087, 179985, 225455, 215003, 148677, 73868, 218125, 263479, 270476, 173121, 209717, 188597, 257871]


## Work Cited
> F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>